# References (& Notes)

The `find_peaks` function from `scipy` has an attribute called `prominence` which was a new concept for me in signal processing.

The essence of it is (centred around different local maxima of a function) and sees the distance from the two tangents (that are slightly offset from the peaks) to their lowest point. That distance in amplitude indicates the prominence of the signal and is used in signal processing to *smooth-off* noise.

Lines 122-132:

Below is the code excerpt

In [ ]:
'''
elif (time.time() - poor_posture_start_time) > threshold_time:
    """
    There is now sustained poor posture, therefore model a distribution of the deviances of the posture. Depending on the threshold time, I could include a recency bias.
    Send a signal to serial indicating the position of this poor posture and where the average of this is.
    The signal processing (a dedicated function for ease) will send out:
        1. The sensor(s) which is/are nearest to the deviations
        2. The intensity of the haptic motor signal (from the distribution of peaks (should use deques for this aspect with the recency bias))
        3. Tackle the issue of n-number deviances
        4. Share this to the serial in this format: "serial.write(bf'Curve, {signal_response[0]}, {signal_response[1]}')"
        5. Use the indices to plot the points in 3D space
    """
'''

It has been taking me much longer to solve this issue that I have since there are multiple overlapping requirements that need to be handled by some helper functions. Therefore, I will take the time to explain my thinking process for this and include links to subsequent files in this repo the explores the derivation for this solution in more depth.

#### Rolling Mean
For the duration of the *deviation* there will be a distribution of different functions (deviation against postition along spine graph) for each instance of this. To aggregate this, I would like to make an algorithm to make a mean that can account for anomalies (I don't think any of the measures of central tendency accounts for this specific goal that I have).

Below will explore this clearer:
$$
\mathrm{devation\_array\_0} = \begin{bmatrix}i_0 & i_1 & i_2 & ... & i_n\end{bmatrix} \\\

\\
\mathrm{devation\_array\_1} = \begin{bmatrix}j_0 & j_1 & j_2 & ... & j_n\end{bmatrix} \\\

\\
... \\\

\\
\mathrm{devation\_array\_n} = \begin{bmatrix}k_0 & k_1 & k_2 & ... & k_n\end{bmatrix}
$$

For each column in this large matrix, I would like to create an average of the results to output a final array which can be approximated by a single function:$\mathrm{aggregated\_devation\_array} = \begin{bmatrix}\lambda_0 & \lambda_1 & \lambda_2 & ... & \lambda_n\end{bmatrix}$, which approximates to $f(x)$.

However, if there were anomalous extremes in this (the aggregation of noise in the sensors for examples or just a random spike) I wouldn't want this affecting, drastically, the distribution of the points and deviation.

Therefore, this algorithm will occur with an example:

Let's define the list:
$$\gamma = [3,5,6,9,13,15,1700]$$
The arithmetic mean of this distribution is: $$\frac{1751}{7} \approx 250$$
If using groups of length 4 however, the result differ: $$\frac{\frac{3+5+6+9}{4} + \frac{5+6+9+13}{4} + \frac{6+9+13+15}{4} + \frac{9+13+15+1700}{4}}{4}\\\
\\=\frac{459}{4} \approx 115$$

I am not sure why this result arises but trying to test and define this algorithm will help with my application in code.

A better approach for summarising the values of the $n$-th index could be centering the values around the median value and then finding the interquartile mean. From this, it would help to remove the anomalous values: however, through the testing below it could be shown that this approach actually removes important areas (higher values than the mean doesn't always mean anomalous!)

In [ ]:
import numpy as np


"""
List Setup
"""

# use a random list of varying degrees as the base test
array_1 = np.random.uniform(0, 100, 200)
array_1 = np.concatenate([array_1, np.random.uniform(0,100,5)])

# then use a similar list but with a very large values placed as anomalies
anomalies = np.random.uniform(10000,25000,5)
array_2 = np.concatenate([array_1, anomalies])

# finally use a fluctuating list

# try to plot each of these values against each other and choose the best appraoch to this (with my data)

def listing_method(n, list_input):
    """
    There is an error that if I input a value of "n" larger than len(list_input) after one error has occurred, it will just default to time_series_mean == 0.000
    I do not know how to to debug that.
    """
    setup = True
    while setup:
        if n > len(list_input) or n < 1 or not isinstance(n, int):
            try:
                n = int(input(f"Enter a value for n in range 1 and {len(list_input)}: "))
            except ValueError:
                print("Please enter an integer.")
        else:
            setup = False  # exit only when n is valid


    """
    Input the algorithm for this.
    """
    list_input = np.array(list_input)
    subdivided_lists = [list_input[i:(i+n)] for i in range((len(list_input)) - n + 1)]
    subdivided_list_mean = [sum(subdivided_lists[i])/n for i in range(len(subdivided_lists))]
    time_series_mean = np.mean(subdivided_list_mean)
    return time_series_mean

def interquartile_mean_calculation(poor_posture_list_input):
    list_input = np.array(poor_posture_list_input)
    n = len(list_input)

    if n == 0:
        return np.nan # an empty list
    if n < 4: # not enough elements for any meaningful IQR
        return np.mean(list_input)

    list_input.sort()
    n_upper = int(n * 0.75)
    n_lower = int(n * 0.25)

    interquartile_list = list_input[n_lower:n_upper]

    if interquartile_list.size == 0: # incase an unexpected error occurred
        return np.mean(list_input)

    interquartile_mean = np.mean(interquartile_list)
    return interquartile_mean

n = 15  # how can I numerically find the best value for n, given the list size?
i = listing_method(n, array_1)
j = listing_method(n, array_2)
k = interquartile_mean(array_1)
p = interquartile_mean(array_2)

med1 = np.median(array_1)
med2 = np.median(array_2)

print(f"listing, array_1: {i}")
print(f"listing, array_2: {j}")
print(f"IQM, array_1: {k}")
print(f"IQM, array_2: {p}")
print(f"Median, array_1: {med1}\nMedian, array_2: {med2}")

From running this, the interquartile method is significantly better since the runtime is much, much lower (and this will have to run in real time).

I will look into ways of optimising this algorithm but since it is dependent on length of the list (of size n) then accuracy will be lost through this process.

I'll try to run other similar methods as well to check but the interquartile method is the best (and I will adjust the size of that window (10-90 or 40-60 depending on what I am looking for) in the actual code).

I am now interested in how I can find the optimal value for this (if I run some tests) to see what would actually be the best window size - if such a thing exists.

In [ ]:
import numpy as np

lis = [9,9,9,8,8]
l = np.array(lis)
q = np.array(l)
print(l)
print(q)


Curvature test code

In [ ]:
"""
Cubic spline test: here is a test for the cubic spline calculation since my previous endeavours were actually quite messy (and unhelpful).
I will now apply this to the IMU sensor test.
"""
from scipy.interpolate import CubicSpline
import numpy as np
import matplotlib.pyplot as plt

fig= plt.figure(figsize=(15,8))
ax = fig.add_subplot(121, projection='3d')


a = (5,4,2)
b = (6,7,5)
c = (7,9,6)
array = np.array([np.array(i) for i in (a,b,c)])

t = [0]
c_dist = [0]
d_dist = 0
for i in range(len(array)):
    if i == len(array)-1:
        break

    distance = np.linalg.norm(array[i+1]-array[i])
    c_dist.append(distance)
    t.append(sum(c_dist))

x = array[:, 0]
y = array[:, 1]
z = array[:, 2]

xc = CubicSpline(t,x)
yc = CubicSpline(t,y)
zc = CubicSpline(t,z)

plotting_t = np.linspace(min(t), max(t), 1000)

ax.plot(xc(plotting_t), yc(plotting_t), zc(plotting_t))

ax.set_xlim(min(xc(plotting_t)), max(xc(plotting_t)))
ax.set_ylim(min(yc(plotting_t)), max(yc(plotting_t)))
ax.set_zlim(min(zc(plotting_t)), max(zc(plotting_t)))
ax.grid()

"""
Curvature Calculations
"""
curvature_list = []
for i in range(len(plotting_t)):
    r = (xc(plotting_t[i], 0), yc(plotting_t[i], 0), zc(plotting_t[i], 0))
    r_prime = (xc(plotting_t[i], 1), yc(plotting_t[i], 1), zc(plotting_t[i], 1))
    r_double_prime = (xc(plotting_t[i], 2), yc(plotting_t[i], 2), zc(plotting_t[i], 2))

    kappa = (np.linalg.norm(np.cross(r_prime, r_double_prime)))/(np.linalg.norm(r_prime)**3)
    curvature_list.append(kappa)

ax2 = fig.add_subplot(122)
ax2.plot(plotting_t, curvature_list)
ax2.grid()
ax2.set_aspect(250)

ax.set_title("Parametric Curve")
ax2.set_title(r"d$\kappa$/dt")

plt.tight_layout()
plt.show()

**Curvature Derivation & Practice Questions**:

I want to intuitively explain what this means (understanding helps with adding additional features) so I will copy up workings out into LaTeX, make apparent any errors in my process and then apply this to the IMU code as normal.

In this derivation, review circular motion and see how it relates to this (and do practice questions on it since my physics is worsening). Try out some PAT, TMUA and MAT papers.

Once this is done, the coded aspect of my project is entirely finished off!!

----------

**Curvature Application**: Here is how I would like to apply this (*eventually it will just be machine learning but we have not gotten there yet*):

- There will be a list of the instaneous curvatures along the spline
- There will be a storage of how those curvatures change wrt time
- There will be a maximum (that is tracked (and maybe marked with a red dot))
- There may be a measure of the standard deviation of points
- There may be an average (mean/median)

*For the initial application, I will be using max($\kappa$) as it is easiest (I hope) to get working. Then, I will try to develop a pseudocode and apply updates incrementally that works towards the main goal (as written below)*:

    
    When a user wears the posture bracing, there will be a calibration phase at the beginning where there is an implied best posture. They will then do a range of exercises that will help the model "understand" what the user's spine shape is. Then, the threshold is set on the deviance away from this ideal shape.

To scaffold onto this ideal, I think it will be best to go as follows:

1. max($\kappa$)
2. how $\kappa$ changes with time; larger differences indicates big curves (if the change results in a larger value -- *this will be difficult code to make robust so maybe this won't be second*)
3. integrating across the whole spine and then dividing by the change in bounds
    

**Derivation**:
https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/curvature/v/curvature-formula-part-5

When trying to find

In [ ]:
from collections import deque
import numpy as np

deck = deque(maxlen=10)
deck.extend([3, 9, 9, 190])
d = np.mean(deck)
e = np.median(deck)
print(e)

In [ ]:
import time

t = time.time()
print(t)
time.sleep(2)
for i in range(8):
    for i in range(9):
        q = time.time()
        time.sleep(1.2)
        z = time.time()
        print(q - z)
        print(q - t)

print(t)
print(q)
print(t-q)

Below is the error in the code:


    ```python
        if IMU_ID == 1:
            IMU_DEQUES[0].append(IMU_DATA_NORM)

        elif IMU_ID == 2:
            IMU_DEQUES[1].append(IMU_DATA_NORM)

        elif IMU_ID == 7:
            IMU_DEQUES[2].append(IMU_DATA_NORM)
    ```

Which means that this cannot be generalised.

All I need to do is export, through the serial and handle it, which sensor IDs are successful and place them into a list such that the improved one will be:

    ```python
    try:
        target_IMU_index = IMU_ID_LIST.index(IMU_ID)
        IMU_DEQUES[target_IMU_index].append(IMU_DATA_NORM)

    except ValueError:
        print(f"Warning: Recieved data from unknown data channel @ ID:{IMU_ID}")

    ```